# Caplet Pricing Using Gaussian Quadrature

This notebook implements caplet pricing formulas using Gaussian quadrature methods, focusing on Risk-Free Rate (RFR) Modeling with Average Rate Caplet Pricing.

In [ ]:
# Module imports and setup
import sys
import os
import importlib
import numpy as np
import matplotlib.pyplot as plt
import jax
import jax.numpy as jnp
import plotly.express as px
import plotly.graph_objects as go
from scipy.stats import norm
from plotly.subplots import make_subplots

# Add project root to Python path to ensure imports work correctly
project_root = os.path.abspath(os.path.join(os.getcwd(), ".."))
if project_root not in sys.path:
    sys.path.append(project_root)

# Direct imports from module paths
from src.gaussian_quadrature.utils import quadrature as quad
from src.gaussian_quadrature.models import bachelier as bach

# Function to reload modules
def reload_modules():
    importlib.reload(quad)
    importlib.reload(bach)
    print("Modules reloaded successfully!")

# Run this when you need to refresh the modules
reload_modules()

Modules reloaded successfully!


## 1. Introduction to Caplets

Caplets are interest rate options that provide protection against rising interest rates. They are essentially call options on interest rates, giving the holder the right but not the obligation to benefit from an interest rate that is higher than a specified strike rate.

### Mathematical Definition

A caplet with strike rate $K$, notional amount $N$, and covering the period from time $T_1$ to $T_2$ has a payoff at time $T_2$ of:

$$\text{Payoff}_{T_2} = N \cdot \tau \cdot \max(L(T_1, T_2) - K, 0)$$

Where:
- $L(T_1, T_2)$ is the forward rate set at time $T_1$ for the period $[T_1, T_2]$
- $\tau$ is the year fraction between $T_1$ and $T_2$ (typically $T_2 - T_1$)
- $N$ is the notional amount
- $K$ is the strike rate

## 2. The Bachelier Model

In the Bachelier model, interest rates are assumed to follow an arithmetic Brownian motion, which allows for negative interest rates (unlike the lognormal models). This is particularly relevant in today's low interest rate environment.

### Model Dynamics

Under the Bachelier model, the forward rate $L(t, T_1, T_2)$ follows:

$$dL(t, T_1, T_2) = \sigma dW_t$$

where:
- $\sigma$ is the volatility parameter
- $W_t$ is a standard Brownian motion under the appropriate measure

At time $T_1$, the distribution of the forward rate is normal:

$$L(T_1, T_1, T_2) \sim \mathcal{N}\left(L(0, T_1, T_2), \sigma^2 T_1\right)$$

## 3. Caplet Pricing Formula

The price of a caplet at time $t=0$ under the Bachelier model can be expressed as:

$$\text{Caplet}(0) = P(0, T_2) \cdot N \cdot \tau \cdot \mathbb{E}_{T_2}[\max(L(T_1, T_2) - K, 0)]$$

Where $P(0, T_2)$ is the discount factor from time 0 to $T_2$, and $\mathbb{E}_{T_2}$ represents the expectation under the $T_2$-forward measure.

### Analytical Formula

The caplet price has a closed-form solution under the Bachelier model:

$$\text{Caplet}(0) = P(0, T_2) \cdot N \cdot \tau \cdot [(L(0, T_1, T_2) - K) \cdot \Phi(d) + \sigma \sqrt{T_1} \cdot \phi(d)]$$

where:
- $\Phi(\cdot)$ is the standard normal cumulative distribution function
- $\phi(\cdot)$ is the standard normal probability density function
- $d = \frac{L(0, T_1, T_2) - K}{\sigma \sqrt{T_1}}$

### Gaussian Quadrature Approach

While the analytical formula provides an exact solution, we can also use Gaussian quadrature to price the caplet by evaluating the following integral:

$$\text{Caplet}(0) = P(0, T_2) \cdot N \cdot \tau \cdot \int_{-\infty}^{\infty} \max(F + \sigma\sqrt{T_1}x - K, 0) \frac{1}{\sqrt{2\pi}} e^{-\frac{x^2}{2}} dx$$

where $F = L(0, T_1, T_2)$ is the initial forward rate.

In [12]:
def caplet_price_analytical(F, K, sigma, T1, tau, discount_factor, notional=1.0):
    """Calculate the caplet price using the Bachelier analytical formula.
    
    Args:
        F: Forward rate at time 0 for the period [T1, T2]
        K: Strike rate
        sigma: Volatility
        T1: Time to rate fixing
        tau: Year fraction between T1 and T2 (typically T2 - T1)
        discount_factor: Discount factor P(0, T2)
        notional: Notional amount (default=1.0)
    
    Returns:
        Caplet price
    """
    from scipy.stats import norm
    
    d = (F - K) / (sigma * np.sqrt(T1)) if T1 > 0 and sigma > 0 else float('inf')
    
    if np.isinf(d):
        # Handle edge case
        return discount_factor * notional * tau * max(F - K, 0)
    
    normal_cdf = norm.cdf(d)
    normal_pdf = norm.pdf(d)
    
    # Bachelier formula
    price = discount_factor * notional * tau * (
        (F - K) * normal_cdf + sigma * np.sqrt(T1) * normal_pdf
    )
    
    return price

## 4. Implementing Gaussian Quadrature for Caplet Pricing

We'll now implement the Gaussian quadrature approach to price caplets. We need to transform the integral to match the form suitable for Gaussian-Hermite quadrature, which is designed for integrals of the form:

$$\int_{-\infty}^{\infty} f(x) e^{-x^2} dx \approx \sum_{i=1}^n w_i f(x_i)$$

### Transformation for Gaussian-Hermite Quadrature

For our caplet price integral, the transformation is straightforward as our integrand already includes $e^{-\frac{x^2}{2}}$, which is proportional to $e^{-x^2}$ with a simple change of variable.

In [13]:
def caplet_integrand(x, F, K, sigma, T1):
    """Integrand function for caplet pricing using Gaussian-Hermite quadrature.
    
    Args:
        x: Integration variable
        F: Forward rate at time 0
        K: Strike rate
        sigma: Volatility
        T1: Time to rate fixing
    
    Returns:
        Value of the integrand at point x
    """
    # Calculate the forward rate at time T1
    forward_rate = F + sigma * np.sqrt(T1) * x
    
    # Calculate the payoff
    payoff = np.maximum(forward_rate - K, 0)
    
    # No need to multiply by e^(-x^2/2) as it's handled by the quadrature weights
    return payoff

def caplet_price_quadrature(F, K, sigma, T1, tau, discount_factor, notional=1.0, n_points=20):
    """Calculate the caplet price using Gaussian-Hermite quadrature.
    
    Args:
        F: Forward rate at time 0 for the period [T1, T2]
        K: Strike rate
        sigma: Volatility
        T1: Time to rate fixing
        tau: Year fraction between T1 and T2
        discount_factor: Discount factor P(0, T2)
        notional: Notional amount
        n_points: Number of quadrature points
    
    Returns:
        Caplet price calculated using Gaussian-Hermite quadrature
    """
    from numpy.polynomial.hermite import hermgauss
    
    # Special case for zero volatility or immediate exercise
    if sigma == 0 or T1 == 0:
        return discount_factor * notional * tau * max(F - K, 0)
    
    # Get Gauss-Hermite quadrature points and weights
    x, w = hermgauss(n_points)
    
    # Calculate the integrand at each quadrature point
    integrand_values = caplet_integrand(x, F, K, sigma, T1)
    
    # Perform the weighted sum (adjust for the standard Hermite normalization)
    # The factor 1/sqrt(pi) is because hermgauss uses e^(-x^2) as weight,
    # while the standard normal PDF has e^(-x^2/2)/sqrt(2pi) as weight
    integral = np.sum(w * integrand_values) / np.sqrt(np.pi)
    
    # Calculate the final price
    price = discount_factor * notional * tau * integral
    
    return price

## 5. Comparing Analytical and Quadrature Methods

Let's compare the results of the analytical formula with the Gaussian quadrature method for different parameter sets.

In [14]:
# Set up parameter values
F = 0.02       # Initial forward rate (2%)
K = 0.025      # Strike rate (2.5%)
sigma = 0.004  # Volatility (40 basis points)
T1 = 1.0       # Time to rate fixing (1 year)
T2 = 1.5       # Payment time (1.5 years)
tau = T2 - T1  # Year fraction (0.5 years)
discount_factor = 0.97  # Discount factor P(0, T2)
notional = 1000000  # Notional amount (1 million)

# Calculate price using both methods
price_analytical = caplet_price_analytical(
    F, K, sigma, T1, tau, discount_factor, notional
)

# Try different numbers of quadrature points
quad_points = [5, 10, 15, 20, 30, 50]
prices_quadrature = []

for n in quad_points:
    price_quad = caplet_price_quadrature(
        F, K, sigma, T1, tau, discount_factor, notional, n_points=n
    )
    prices_quadrature.append(price_quad)

# Display results
print(f"Caplet Price (Analytical): {price_analytical:.2f}")

for i, n in enumerate(quad_points):
    print(f"Caplet Price (Quadrature, n={n}): {prices_quadrature[i]:.2f}, "  
          f"Difference: {prices_quadrature[i] - price_analytical:.6f}")

# Visualize convergence
fig = go.Figure()
fig.add_trace(go.Scatter(
    x=quad_points,
    y=[abs(price - price_analytical) for price in prices_quadrature],
    mode='lines+markers',
    name='Absolute Error',
    line=dict(color='blue', width=2)
))

fig.update_layout(
    title="Convergence of Gaussian Quadrature Method",
    xaxis_title="Number of Quadrature Points",
    yaxis_title="Absolute Error",
    yaxis_type="log",  # Log scale for better visualization of error
    width=800,
    height=500
)

fig.show()

Caplet Price (Analytical): 98.14
Caplet Price (Quadrature, n=5): 16.82, Difference: -81.318210
Caplet Price (Quadrature, n=10): 20.69, Difference: -77.447744
Caplet Price (Quadrature, n=15): 19.30, Difference: -78.842942
Caplet Price (Quadrature, n=20): 17.24, Difference: -80.901009
Caplet Price (Quadrature, n=30): 22.85, Difference: -75.285046
Caplet Price (Quadrature, n=50): 22.12, Difference: -76.014045


## 1. Risk-Free Rate (RFR) Modeling with Average Rate Caplet Pricing

This section implements a comprehensive RFR framework, incorporating both compound and arithmetic average term rates.

### 1.1 Extended T-Forward Measure

The extended T-forward measure uses a bank account (also called money-savings account) numeraire. This is defined as:

$$dB_t = r_t B_t dt, \quad B_t = \exp\left(\int_0^t r_u du\right)$$

where $r_t$ represents the short rate.

The extended zero-coupon bond can be expressed as:

$$P_{t,T} = 
\begin{cases} 
\mathbb{E}_t\left[\exp\left(-\int_{t,T} r_u du\right)\right], & t \leq T \\
\exp\left(-\int_{t,T} r_u du\right) = \frac{B_t}{B_T}, & t > T
\end{cases}$$

The associated risk-neutral measure, denoted $\mathbb{Q}^T$, corresponds to the T-forward measure for $t \leq T$ and the money-savings measure after maturity.

### 1.2 Compound and Arithmetic Average Term Rates

For an observation period $[T-\tau, T]$, we can define two types of rates:

1. **Compound rates**: The daily-compounded setting-in-arrears rate is defined as:

   $$R_T := R(T-\tau, T) = \frac{1}{\tau}\left[\prod_{d\in\mathcal{B}(T-\tau,T)} (1 + \tau_d r_d) - 1\right] \approx \frac{1}{\tau}\left[\exp\left(\int_{T-\tau}^T r_u du\right) - 1\right]$$

   where $\mathcal{B}(T-\tau, T)$ runs over business days in $[T-\tau, T]$.

2. **Arithmetic average rates**: The daily arithmetic average setting-in-arrears rate is:

   $$A_T := A(T-\tau, T) = \frac{1}{\tau}\left[\sum_{d\in\mathcal{B}(T-\tau,T)} \tau_d r_d\right] \approx \frac{1}{\tau}\int_{T-\tau}^T r_u du$$

   This can also be approximated as:

   $$A_T \approx \frac{1}{\tau} \log(1 + \tau R(T-\tau, T))$$

It's important to note that the compound rates implied by $A_T$ might differ from actual quotes for $R_T$ if those markets are distinct, due to basis.

### 1.3 Backward-Looking Forward Rates

In the extended T-forward measure, we can express the compound forward rate as:

$$R_{t,T} := R_t(T-\tau, T) = \mathbb{E}_{t,T}[R(T-\tau, T)]$$

For different time periods:
- For $t \leq T-\tau$: $R_{t,T} = \frac{1}{\tau}\left(\frac{P_{t,T-\tau}}{P_{t,T}} - 1\right)$ is analogous to LIBOR.
- For $T-\tau < t \leq T$: $R_{t,T} = \frac{1}{\tau}\left(\frac{B_t}{B_{T-\tau}P_{t,T}} - 1\right)$ is daily compounding.
- Post maturity: $R_{t,T} \equiv \frac{1}{\tau}\left(\frac{B_T}{B_{T-\tau}} - 1\right)$ is known and fixed.

The yield curve $T \rightarrow P_{t,T}$ can be bootstrapped from $T \rightarrow R_{t,T}$ quotes.

The arithmetic average forward rate can be expressed as:

$$A_{t,T} := A_t(T-\tau, T) = \mathbb{E}_{t,T}\left[\frac{1}{\tau}\log(1 + \tau R(T-\tau, T))\right]$$

Since the payoff is non-linear, $A_{t,T}$ is model dependent. A convexity correction can appear compared to market quotes. Moreover, in case of non-trivial basis, the spread between risk-free valuation and market can widen.

### 1.4 Caplet Pricing with Different Rate Types

Using $P_{t,T}$ as numeraire, for compound rates:

$$V_t^{\text{Caplet}}[R_T, K] = P_{t,T}\mathbb{E}_{t,T}[\tau(R(T-\tau, T) - K)^+]$$

For arithmetic average rates, the choice of cutoff $g_{t,T}$ influences directly the convexity correction. We should attempt to choose it such that:

$$V_t^{\text{Caplet}}[A_T, K] = P_{t,T}\mathbb{E}_{t,T}[\tau(A(T-\tau, T) - K)^+]$$

$$= P_{t,T}\mathbb{E}_{t,T}[(\log(1 + \tau R(T-\tau, T)) - \tau K)^+]$$

Another approach, taken by Hasegawa (2021), is to use a short-rate model for $r_t$ working in money-savings numeraire which treats $R_{t,T}$ and $A_{t,T}$ in a single model (the author neglects basis, but it can be added as an additional process).

In [ ]:
# Implementation of caplet pricing with average rates using the framework described

def compound_rate_caplet_price(tau, K, Rt, sigma, T1, PtT, n_points=20):
    """
    Calculate the price of a caplet based on compound rates.
    
    Args:
        tau: Year fraction (accrual period)
        K: Strike rate
        Rt: Forward rate
        sigma: Volatility parameter
        T1: Time to rate fixing
        PtT: Discount factor P(0,T2)
        n_points: Number of quadrature points
        
    Returns:
        Price of the compound rate caplet
    """
    # For compound rates, we can use the standard Bachelier formula
    return bach.bachelier_caplet_price_laguerre(tau, K, Rt, sigma, PtT, n_points)

def average_rate_caplet_price(tau, K, Rt, sigma, T1, T2, PtT, n_points=20):
    """
    Calculate the price of a caplet based on arithmetic average rates.
    
    Args:
        tau: Year fraction (accrual period)
        K: Strike rate
        Rt: Forward rate
        sigma: Volatility parameter
        T1: Start of averaging period
        T2: End of averaging period (T1 + tau)
        PtT: Discount factor P(0,T2)
        n_points: Number of quadrature points
        
    Returns:
        Price of the average rate caplet
    """
    # Adjust the volatility for average rate effect
    # Variance of the average rate under Bachelier model
    var_avg = (sigma**2 * T1 / 3) * (1 + (T2-T1)/(2*T1))
    sigma_avg = np.sqrt(var_avg)
    
    # Use adjusted volatility in the standard Bachelier pricing formula
    return bach.bachelier_caplet_price_laguerre(tau, K, Rt, sigma_avg, PtT, n_points)

# Implement the log transformation approach from Hasegawa (2021)
def log_transform_average_rate_caplet_price(tau, K, Rt, sigma, T1, T2, PtT, n_points=20):
    """
    Calculate the price of a caplet on arithmetic average rates using log transformation.
    
    This implements the approach where:
    V_t^Caplet[A_T, K] = P_{t,T}E_{t,T}[(log(1 + tau*R(T-tau, T)) - tau*K)^+]
    
    Args:
        tau: Year fraction (accrual period)
        K: Strike rate
        Rt: Forward rate
        sigma: Volatility parameter
        T1: Start of averaging period
        T2: End of averaging period (T1 + tau)
        PtT: Discount factor P(0,T2)
        n_points: Number of quadrature points
        
    Returns:
        Price of the average rate caplet using log transformation
    """
    # Define the log-transformed integrand
    def log_transform_integrand(y, Rt, K, sigma, T1, tau, d):
        # Calculate standardized variable and payoff
        x = np.sqrt(y) + d
        log_payoff = np.maximum(np.log(1 + tau * (Rt + sigma * np.sqrt(T1) * x)) - tau * K, 0)
        
        # Calculate PDF and Jacobian
        pdf = np.exp(-x**2/2) / np.sqrt(2*np.pi)
        jacobian = 1 / (2 * np.sqrt(y))
        
        return log_payoff * pdf * jacobian
    
    # Calculate standardized moneyness
    log_K = np.log(1 + tau * K) / tau
    log_F = np.log(1 + tau * Rt) / tau
    
    # Adjust volatility for log transformation
    # This is an approximation - in practice would need more detailed model calibration
    vol_adjust = 1 / (1 + tau * Rt)
    sigma_log = sigma * vol_adjust
    
    # Calculate standardized moneyness for the log-transform
    d = (log_F - log_K) / (sigma_log * np.sqrt(T1))
    
    from functools import partial
    integrand = partial(
        log_transform_integrand, 
        Rt=Rt, K=K, sigma=sigma, T1=T1, tau=tau, d=d
    )
    
    # Use Gauss-Laguerre quadrature
    integral, error = quad.gauss_laguerre_quadrature_eigenvalue(integrand, n_points)
    
    # Calculate final price
    price = PtT * integral
    
    return price

### 1.5 Analyzing Basis and Convexity Effects

The difference between arithmetic average rates and compound rates creates a basis that affects option prices. Additionally, the non-linearity of average rate caplet payoffs introduces convexity corrections.

Let's analyze these effects across different strike levels and maturities:

In [ ]:
# Analysis of basis and convexity effects across different strikes

# Set base parameters
tau = 0.5
sigma = 0.004
T1 = 1.0
T2 = T1 + tau
PtT = 0.97
Rt = 0.02  # Base forward rate

# Create range of strike rates
strike_rates = np.linspace(0.01, 0.03, 20)  # 1% to 3%

compound_prices = []
average_prices = []
log_prices = []

for K in strike_rates:
    compound_prices.append(compound_rate_caplet_price(tau, K, Rt, sigma, T1, PtT))
    average_prices.append(average_rate_caplet_price(tau, K, Rt, sigma, T1, T2, PtT))
    log_prices.append(log_transform_average_rate_caplet_price(tau, K, Rt, sigma, T1, T2, PtT))

# Calculate price differences and ratios
avg_compound_diff = np.array(average_prices) - np.array(compound_prices)
log_compound_diff = np.array(log_prices) - np.array(compound_prices)
price_ratios = np.array(average_prices) / np.array(compound_prices)
log_ratios = np.array(log_prices) / np.array(compound_prices)

# Create visualization
fig = make_subplots(rows=2, cols=1, 
                    subplot_titles=("Caplet Prices by Strike Rate", 
                                   "Ratio of Average Rate to Compound Rate Prices"),
                    row_heights=[0.6, 0.4],
                    shared_xaxes=True,
                    vertical_spacing=0.1)

# Add price curves
fig.add_trace(
    go.Scatter(x=strike_rates*100, y=compound_prices, 
              name="Compound Rate", line=dict(color="blue")),
    row=1, col=1
)
fig.add_trace(
    go.Scatter(x=strike_rates*100, y=average_prices, 
              name="Avg Rate (Vol Adj)", line=dict(color="red")),
    row=1, col=1
)
fig.add_trace(
    go.Scatter(x=strike_rates*100, y=log_prices, 
              name="Avg Rate (Log Transform)", line=dict(color="green")),
    row=1, col=1
)

# Add vertical line at forward rate
fig.add_shape(
    type="line", x0=Rt*100, x1=Rt*100, y0=0, y1=max(compound_prices)*1.1,
    line=dict(color="gray", dash="dash"), row=1, col=1
)
fig.add_annotation(
    x=Rt*100, y=max(compound_prices)*0.8,
    text=f"Forward Rate: {Rt*100:.1f}%",
    showarrow=True, arrowhead=1, row=1, col=1
)

# Add price ratio curves
fig.add_trace(
    go.Scatter(x=strike_rates*100, y=price_ratios, 
              name="Vol Adj / Compound", line=dict(color="orange")),
    row=2, col=1
)
fig.add_trace(
    go.Scatter(x=strike_rates*100, y=log_ratios, 
              name="Log / Compound", line=dict(color="purple")),
    row=2, col=1
)

# Add horizontal line at ratio=1
fig.add_shape(
    type="line", x0=min(strike_rates)*100, x1=max(strike_rates)*100, 
    y0=1, y1=1, line=dict(color="black", dash="dash"), row=2, col=1
)

# Update layout
fig.update_layout(
    title="Comparison of Compound vs. Average Rate Caplet Pricing",
    xaxis2_title="Strike Rate (%)",
    yaxis_title="Caplet Price",
    yaxis2_title="Price Ratio",
    legend=dict(orientation="h", yanchor="bottom", y=1.02, xanchor="right", x=1),
    height=700, width=800
)

fig.show()

In [ ]:
# Analysis of maturity effects

# Fix strike rate at ATM level
K = 0.02  # At-the-money (equal to forward rate)
Rt = 0.02
sigma = 0.004
tau = 0.5
PtT = 0.97

# Create range of T1 values (time to start of averaging period)
T1_values = np.linspace(0.25, 3.0, 12)  # 3 months to 3 years

compound_prices_T = []
average_prices_T = []
log_prices_T = []

for T1 in T1_values:
    T2 = T1 + tau
    compound_prices_T.append(compound_rate_caplet_price(tau, K, Rt, sigma, T1, PtT))
    average_prices_T.append(average_rate_caplet_price(tau, K, Rt, sigma, T1, T2, PtT))
    log_prices_T.append(log_transform_average_rate_caplet_price(tau, K, Rt, sigma, T1, T2, PtT))

# Calculate price ratios
price_ratios_T = np.array(average_prices_T) / np.array(compound_prices_T)
log_ratios_T = np.array(log_prices_T) / np.array(compound_prices_T)

# Create visualization
fig = make_subplots(rows=2, cols=1, 
                    subplot_titles=("ATM Caplet Prices by Maturity", 
                                   "Ratio of Average Rate to Compound Rate Prices"),
                    row_heights=[0.6, 0.4],
                    shared_xaxes=True,
                    vertical_spacing=0.1)

# Add price curves
fig.add_trace(
    go.Scatter(x=T1_values, y=compound_prices_T, 
              name="Compound Rate", line=dict(color="blue")),
    row=1, col=1
)
fig.add_trace(
    go.Scatter(x=T1_values, y=average_prices_T, 
              name="Avg Rate (Vol Adj)", line=dict(color="red")),
    row=1, col=1
)
fig.add_trace(
    go.Scatter(x=T1_values, y=log_prices_T, 
              name="Avg Rate (Log Transform)", line=dict(color="green")),
    row=1, col=1
)

# Add price ratio curves
fig.add_trace(
    go.Scatter(x=T1_values, y=price_ratios_T, 
              name="Vol Adj / Compound", line=dict(color="orange")),
    row=2, col=1
)
fig.add_trace(
    go.Scatter(x=T1_values, y=log_ratios_T, 
              name="Log / Compound", line=dict(color="purple")),
    row=2, col=1
)

# Add horizontal line at ratio=1
fig.add_shape(
    type="line", x0=min(T1_values), x1=max(T1_values), 
    y0=1, y1=1, line=dict(color="black", dash="dash"), row=2, col=1
)

# Update layout
fig.update_layout(
    title="Maturity Effect on Compound vs. Average Rate Caplet Pricing (ATM Strike)",
    xaxis2_title="Time to Start of Averaging Period (Years)",
    yaxis_title="ATM Caplet Price",
    yaxis2_title="Price Ratio",
    legend=dict(orientation="h", yanchor="bottom", y=1.02, xanchor="right", x=1),
    height=700, width=800
)

fig.show()

### 1.6 Summary of RFR Modeling for Average Rate Caplet Pricing

The extension to RFR modeling with average rate caplet pricing reveals several important insights:

1. **Convexity Correction**: Arithmetic average rates introduce a convexity correction compared to compound rates, generally resulting in lower caplet prices for average rate-based products.

2. **Volatility Scaling**: The volatility adjustment approach provides a simple but effective way to account for the averaging effect, with the variance of the average rate being approximately $\frac{1}{3}$ of the spot rate variance for typical parameters.

3. **Term Structure Effects**: The difference between compound and average rate caplets varies with maturity, with longer maturities showing more pronounced differences due to accumulated volatility effects.

4. **Log Transform Approach**: Hasegawa's (2021) log transformation method provides an alternative pricing approach that can better account for the non-linearity in average rate contracts, especially for longer maturities or higher volatilities.

5. **Implementation Considerations**: From an implementation standpoint, the volatility-adjusted approach offers the simplest modification to existing compound rate caplet pricing infrastructure, while the log transform approach may require more extensive changes.

The quadrature methods implemented in this project offer efficient and accurate numerical integration for both compound and average rate caplet pricing, with the eigenvalue-based Gauss-Laguerre approach providing excellent stability and performance.